In [1]:
print("hello")

hello


In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForQuestionAnswering

c:\Users\Shehan\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Step 1: Load the CSV file
df = pd.read_csv('ChatData.csv')

In [4]:
# Print the first 5 rows of the DataFrame
print(df.head())

# Print the shape of the DataFrame (number of rows and columns)
print(df.shape)

# Print the column names of the DataFrame
print(df.columns)

# Print the data types of each column in the DataFrame
print(df.dtypes)

# Print summary statistics for the numeric columns in the DataFrame
print(df.describe())

                                            Question   
0               are you a fan of google or microsoft  \
1  im not a huge fan of google but i use it a lot...   
2  yeah their services are good im just not a fan...   
3  did you know google had hundreds of live goats...   
4  i like google chrome do you use it as well for...   

                                              Answer  
0  both are excellent technology they are helpful...  
1  google provides online related services and pr...  
2  google is leading the alphabet subsidiary and ...  
3  it is very interesting google provide chrome o...  
4  yes google is the biggest search engine and go...  
(159719, 2)
Index(['Question', 'Answer'], dtype='object')
Question    object
Answer      object
dtype: object
              Question   Answer
count           159719   159706
unique          110904   156688
top     hi how are you  you too
freq               353      111


In [5]:
print(df['Question'].isnull().sum())
print(df['Answer'].isnull().sum())

0
13


In [6]:
df = df[df['Answer'] != 0]
# Drop rows with null values
df.dropna(inplace=True)

In [7]:
print(df['Question'].isnull().sum())
print(df['Answer'].isnull().sum())

0
0


In [8]:
import re

def clean_text(text):
    # Remove any URLs from the text
    text = re.sub(r'http\S+', '', text)
    
    # Remove any non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    
    # Convert text to lowercase
    text = text.lower()
    
    return text.strip()


In [9]:
print(df['Question'].isnull().sum())
print(df['Answer'].isnull().sum())

0
0


In [10]:
# Step 2: Preprocess the text data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def preprocess_text(text):
    cleaned_text = clean_text(text) # implement a function to clean the text
    tokenized_text = tokenizer.encode(cleaned_text, add_special_tokens=True)
    return tokenized_text

c:\Users\Shehan\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shehan\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:
df['tokenized_question'] = df['Question'].apply(preprocess_text)
df['tokenized_answer'] = df['Answer'].apply(preprocess_text)

In [12]:
# Step 3: Split the data into training and testing sets
train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

In [13]:
# Step 4: Create a Transformer architecture
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [52]:
# Step 5: Train the Transformer model
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

for epoch in range(5):
    for idx, row in train_data.iterrows():
        input_ids = torch.tensor(row['tokenized_question']).unsqueeze(0) # add an extra dimension
        attention_mask = torch.tensor([1] * len(row['tokenized_question'])).unsqueeze(0) # add an extra dimension
        start_positions = torch.tensor([row['tokenized_answer'][0]])
        end_positions = torch.tensor([row['tokenized_answer'][1]])
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


KeyboardInterrupt: ignored

In [54]:
# Step 6: Evaluate the performance of the model
with torch.no_grad():
    for batch in test_data:
        input_ids = torch.tensor(row['tokenized_question']).unsqueeze(0) # add an extra dimension
        attention_mask = torch.tensor([1] * len(row['tokenized_question'])).unsqueeze(0) # add an extra dimension
        start_positions = torch.tensor(batch['tokenized_answer'][0])
        end_positions = torch.tensor(batch['tokenized_answer'][1])
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        predictions_start = torch.argmax(outputs.start_logits, dim=1)
        predictions_end = torch.argmax(outputs.end_logits, dim=1)
        accuracy_start = (predictions_start == start_positions).sum() / len(predictions_start)
        accuracy_end = (predictions_end == end_positions).sum() / len(predictions_end)
        accuracy = (accuracy_start + accuracy_end) / 2

TypeError: ignored